In [2]:
import requests
from bs4 import BeautifulSoup

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
}

In [53]:
class BBCGoodfoodScraper():
    recipes = {}
    def __init__(self):
        self.links = []
        self.rootUrl = 'https://www.bbcgoodfood.com'
        self.recipes['recipes'] = {}

    def getRecipesLinksFromPage(self, url):
        page = requests.get(self.rootUrl+url, headers = HEADERS)
        soup = BeautifulSoup(page.content, 'html.parser')

        for recipe in soup.find_all('h3', class_='teaser-item__title'):
            for link in recipe.find_all('a'):
                self.links.append(link.get('href'));
        nextUrl = soup.find('a', attrs={'title':'Go to next page'})
        try:
            return(nextUrl.get('href'))
        except Exception as inst:
            return None

    def getAllRecipeLinks(self, nextUrl):
        while(nextUrl is not None):
            nextUrl = self.getRecipesLinksFromPage(nextUrl);


    def getRecipe(self, url, idx):
        recipe_title = soup.find('h1', attrs={'class':'recipe-header__title', 'itemprop':'name'}).text
        recipe_details = soup.find('div', attrs={'class':'recipe-details'})
        recipe_ingredients = soup.find('div', attrs={'class':'ingredients-list__content'})
        recipe_method = soup.find('div', attrs={'class':'method'})
        recipe['title'] = recipe_title
        recipe['details']['prep_time'] = recipe_details.find_all('span', attrs={'class':'mins'})[0].text
        recipe['details']['cook_time'] = recipe_details.find_all('span', attrs={'class':'mins'})[1].text
        recipe['details']['difficulty'] = recipe_details.find('span', attrs={'class':'recipe-details__text'}).text
        count = 0
        for ingredient_text in recipe_ingredients.find_all('li', attrs={'class':'ingredients-list__item'}):
            recipe['ingredients'][count] = {}
            recipe['ingredients'][count]['name'] = list(ingredient_text.stripped_strings)[0] if len( list(ingredient_text.stripped_strings)) < 2 else list(ingredient_text.stripped_strings)[0] + ' ' + list(ingredient_text.stripped_strings)[1] 
            count += 1
        recipe['ingredients']['count'] = count
        count = 0
        for method_step in recipe_method.stripped_strings:
            recipe['method'][count] = method_step
            count += 1
        recipe['method']['count'] = count
        self.recipes['recipes'][idx] = recipe

    def getAllRecipes(self):
        count = 0
        for link in self.links:
            self.getRecipe(link, count)
            count += 1
        self.recipes['count'] = count

In [54]:
scraper = BBCGoodfoodScraper()
scraper.getAllRecipeLinks('/recipes/collection/under-20-minutes')

In [55]:
scraper.getAllRecipes()

In [56]:
recipe = {}
rootUrl = 'https://www.bbcgoodfood.com'
url = '/recipes/5856/lemon-chicken-with-spring-veg-noodles'
recipe['details'] = {}
recipe['ingredients'] = {}
recipe['method'] = {}
page = requests.get(rootUrl+url, headers = HEADERS)
soup = BeautifulSoup(page.content, 'html.parser')

In [58]:
scraper.recipes

{'count': 40,
 'recipes': {0: {'details': {'cook_time': '15 mins',
    'difficulty': ' Easy ',
    'prep_time': '5 mins'},
   'ingredients': {0: {'name': '1 tbsp sunflower oil'},
    1: {'name': '2 skinless chicken breasts, cut into strips'},
    2: {'name': 'zest and juice 1 lemon'},
    3: {'name': '1 tbsp caster sugar'},
    4: {'name': '2 tbsp grated fresh root ginger'},
    5: {'name': '2 tsp cornflour'},
    6: {'name': '125g pack or 2 sheets medium dried egg noodle'},
    7: {'name': '200g frozen pea and bean mix'},
    8: {'name': '4 spring onions'},
    9: {'name': '1 tbsp roasted cashews'},
    'count': 10},
   'method': {0: 'Heat the oil in a non-stick pan, then fry the chicken for 5 mins until almost cooked. Tip onto a plate. Pour 250ml water into the pan with the lemon zest and juice, sugar and ginger. In a bowl, mix the cornflour with a little water until smooth, then whisk into the pan. Bring to the boil, stirring, then add the chicken to the sauce. Reduce the heat. Bubb

In [42]:
recipe

{'details': {'cook_time': '15 mins',
  'difficulty': ' Easy ',
  'prep_time': '5 mins'},
 'ingredients': {0: {'name': '1 tbsp sunflower oil'},
  1: {'name': '2 skinless chicken breasts, cut into strips'},
  2: {'name': 'zest and juice 1 lemon'},
  3: {'name': '1 tbsp caster sugar'},
  4: {'name': '2 tbsp grated fresh root ginger'},
  5: {'name': '2 tsp cornflour'},
  6: {'name': '125g pack or 2 sheets medium dried egg noodle'},
  7: {'name': '200g frozen pea and bean mix'},
  8: {'name': '4 spring onions'},
  9: {'name': '1 tbsp roasted cashews'},
  'count': 10},
 'method': {0: 'Heat the oil in a non-stick pan, then fry the chicken for 5 mins until almost cooked. Tip onto a plate. Pour 250ml water into the pan with the lemon zest and juice, sugar and ginger. In a bowl, mix the cornflour with a little water until smooth, then whisk into the pan. Bring to the boil, stirring, then add the chicken to the sauce. Reduce the heat. Bubble for a few mins until chicken is cooked and the sauce th

In [ ]:
#if __name__ == "__main__":
#    scraper = BBCGoodfoodScraper()
#    scraper.getAllRecipeLinks('/recipes/collection/under-20-minutes')
#    scraper.getAllRecipes()
